In [1]:
# Importação das bibliotecas a serem utilizadas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt

In [3]:
# Leitura do nosso arquivo excel (Banco de dados)
file = '/content/Aula 7 - Dados_Desafio_Previsao_Vendas.xls'
df = pd.read_excel(file, parse_dates = True)

#df = pd.read_excel('/content/Aula 7 - Dados_Desafio_Previsao_Vendas.xlsx')
df

FileNotFoundError: ignored

In [ ]:
# Verificar se existe valores nulos no nosso banco de dados

df.info()

In [ ]:
# Verificar os tipos das variáveis

df.dtypes

In [ ]:
# Mostrar detalhes como a média, desvio, mínimo, porcentagem e o máximo

df.describe()

In [ ]:
# Removendo colunas que não serão utilizadas em nossas análises

df1 = df.drop(['VENDEDOR', 'CLIENTE', 'CRP_AJUSTADO', 'UNIDADE_MEDIDA_NF'], axis=1, inplace = True)


In [ ]:
# Fazendo o agrupamento por produto e a soma geral do nosso banco de dados
# Para visualizar a soma da Quantidade e Preço_NF 

n_by_product = df.groupby(["PRODUTO"]).sum()

n_by_product

In [ ]:
# Mudando o tipo para float da coluna 'PRODUTO'

df['PRODUTO'] = df['PRODUTO'].astype(float)

In [ ]:
print(df.groupby('QUANTIDADE')['PRODUTO'].count())

In [ ]:
#Ordenando produto com maior quantidade de vendas

df3 = df.sort_values(by='QUANTIDADE', ascending= False, inplace = True)

In [ ]:
# Pegando pelo ID do vendedor que mais vendeu
df[df.index == 14938]

In [ ]:
# Selecionando apenas o produto '3'

df3 = df.loc[df['PRODUTO'] == 3]

In [ ]:
# Corrigindo a data para ficar no formato de datatime

df['DATA_NF']= pd.to_datetime(df['DATA_NF'], format='%Y-%m-%d', errors='coerce')

In [ ]:
# Plotando o gráfico de calor com a correlação de pearson

plt.rcParams['figure.figsize'] = (15,9)
sns.heatmap(df.corr(method='pearson'), annot=True, fmt='.2f')

In [ ]:
# Ajustando a data numa nova coluna para que fique em Mês e Ano

df['Mes'] = df['DATA_NF'].dt.strftime('%Y-%m')
df_mes = df.groupby(df['Mes'], as_index=False).sum()

df_mes.head()

In [ ]:
# Outra forma de criar uma nova coluna 

# df_mes['mes'] = df['DATA_NF'].apply(lambda x: str(x)[5:7])
# df_mes.head()

In [ ]:
# Criando duas colunas para fazer a nossa predição

series = df_mes[['Mes','QUANTIDADE']]



In [ ]:
series.set_index('Mes', drop=False)

In [ ]:
# Normalizando a coluna Mes para ficar em Mês e Ano e transformando o tipo em Datetime

series['Mes']= pd.to_datetime(series['Mes'], format='%Y-%m', errors='coerce')

series.dtypes

In [ ]:
# Biblioteca para fazer a nossa decomposição

from statsmodels.tsa.seasonal import seasonal_decompose

# Perform additive decomposition
decomp = seasonal_decompose(df_mes['QUANTIDADE'], period=2)

# Plot decomposition
fig = decomp.plot();
fig.set_size_inches((12, 7))
plt.tight_layout();

In [ ]:
# Biblioteca para gerar os gráficos de autocorrelação e autocorrelação parcial

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Create figure and subplot
fig, ax1 = plt.subplots()

# Plot the ACF on ax1
plot_acf(df_mes['QUANTIDADE'], lags=6, zero=False);
plot_pacf(df_mes['QUANTIDADE'], lags=6, zero=False);

In [ ]:
# df = pd.get_dummies(df, columns=['DATA_NF'], drop_first= False)

In [ ]:
series.head()

In [ ]:
series = pd.read_excel(file, parse_dates =['DATA_NF'],index_col=0, squeeze=True)

series['DATA_NF']= pd.to_datetime(series['DATA_NF'], format='%Y-%m', errors='coerce')

# Veja agora o recurso 'Month' é do tipo datetime
series.head()

In [ ]:
series.reset_index(drop=True, inplace = True)

In [ ]:
series1 = series.drop(['CLIENTE', 'PRODUTO', 'CRP_AJUSTADO', 'UNIDADE_MEDIDA_NF', 'PREÇO_NF', 'VLR_PRECO_LISTA', 'VLR_CONTRIBUICAO'], axis=1, inplace = True)


In [ ]:
# series.apply(pd.to_numeric, downcast="integer", errors="ignore")


In [ ]:
series.dtypes


In [ ]:
series.head()

In [ ]:
series.set_index('DATA_NF', inplace=True)

In [ ]:
series.head()
# series = series.set_index('DATA_NF')

In [ ]:
series.index = pd.to_datetime(series.index)
series.dtypes

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing

#Aqui vamos criar três ajustes nos quais tomaremos três valores diferentes 
#de alpha (valores do nível de suavização): 0.2; 0.8 e valor otimizado 
#automaticamente pelo statsmodel que é o recomendado.

fit1 = SimpleExpSmoothing(series).fit(
    smoothing_level=0.2, optimized=False);
fcast1 = fit1.forecast(3).rename(r"$\alpha=0.2$");

fit2 = SimpleExpSmoothing(series).fit(
    smoothing_level=0.8, optimized=False);

fcast2 = fit2.forecast(3).rename(r"$\alpha=0.8$");


fit3 = SimpleExpSmoothing(series).fit()
fcast3 = fit3.forecast(3).rename(r"$\alpha=%s$" % fit3.model.params["smoothing_level"])

plt.figure(figsize=(12, 8));
plt.plot(series, marker="o", color="black");

plt.plot(fit1.fittedvalues, marker="o", color="blue");
(line1,) = plt.plot(fcast1, marker="o", color="blue");
plt.plot(fit2.fittedvalues, marker="o", color="red");
(line2,) = plt.plot(fcast2, marker="o", color="red");

plt.plot(fit3.fittedvalues, marker="o", color="green")
(line3,) = plt.plot(fcast3, marker="o", color="green")

plt.legend([line1, line2, line3], [fcast1.name, fcast2.name, fcast3.name])

In [ ]:
series.dtypes

In [ ]:
print(np.asarray(series))

In [ ]:
type(series.index)